In [4]:
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @chinook.db'

In [6]:
%%sql
Pragma foreign_keys=on;

DROP TABLE if exists Invoice_details;

DROP TABLE if exists Products;

DROP TABLE if exists Invoices;
DROP TABLE if exists Customers;


CREATE TABLE Customers (cust_ID integer NOT NULL Primary key, cust_name varchar, cust_info varchar); 

CREATE TABLE Invoices (invoice_ID integer NOT NULL Primary key,
                    customer_id varchar NOT NULL,
                    order_sum integer,
                    invoice_date integer NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES Customers(cust_ID));

CREATE TABLE Products (product_id integer NOT NULL Primary key,
                       product_name varchar NOT NULL, 
                       price inteder);

CREATE TABLE Invoice_details (invoice_ID integer NOT NULL, 
                            product_id integer NOT NULL,
                            qty integer NOT NULL,
                            primary key (product_id, invoice_id)
                            FOREIGN KEY (product_id) REFERENCES Products(product_id)
                            FOREIGN KEY (invoice_ID) REFERENCES Invoices(invoice_ID));


 * sqlite:///chinook.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [10]:
import sqlite3
db=sqlite3.connect('chinook.db')

In [11]:
type(db)

sqlite3.Connection

In [13]:
cur = db.cursor()

In [27]:
cur.execute("SELECT * FROM Customers")

In [23]:
a = cur.fetchone()

In [24]:
a

(1, 'Microsoft', 'just another OS')

In [26]:
cur = db.cursor()
cur.execute("SELECT * FROM Customers where false")
cur.fetchone()[0]

TypeError: 'NoneType' object is not subscriptable

In [28]:
cur.fetchall()

[(1, 'Microsoft', 'just another OS'), (2, 'Google', 'lmgfy')]

In [30]:

def insert_customer(name, info, db):
    cur = db.cursor()
    cur.execute( '''
    SELECT COALESCE(MAX(cust_ID)+1, 1) FROM Customers''')
    ID = cur.fetchone()[0]
    cur.execute('''
        INSERT INTO Customers(cust_ID, cust_name, cust_info) VALUES(?,?,?)''', (ID,name,info))
    db.commit()

In [31]:
insert_customer('Microsoft', 'just another OS', db)
insert_customer('Google', 'lmgfy', db)

In [32]:
%%sql
select * from customers

 * sqlite:///chinook.db
Done.


cust_ID,cust_name,cust_info
1,Microsoft,just another OS
2,Google,lmgfy
3,Microsoft,just another OS
4,Google,lmgfy


In [184]:
def insert_products(name,price):
    cur=db.cursor()
    cur.execute('''select coalesce(max(product_id)+1,1) from products''')
    id=cur.fetchone()[0]
    cur.execute('''insert into products(product_id, product_name,price) values(?,?,?)''', (id,name,price))
    db.commit()

In [185]:
insert_products('Win10',25000)
insert_products('Google glass', 2000)

In [186]:
%sql select * from products

Done.


product_id,product_name,price
1,Win10,25000
2,Google glass,2000


Написать функцию для добавления заказа. 
Вход параметр_1 - имя клиента, параметр_2 - список продуктов вида [['a',1],['b',2]]
1 шаг - проверка, что есть такие продукты и клиены 
2 шаг - добавление в таблицы invoices and inv-det 

In [187]:
def check_client(name):
    cur = db.cursor()
    cur.execute('''select cust_id from customers where cust_name = ?''', (name,))
    try:
        cli_id = cur.fetchone()[0]
        return cli_id
    except TypeError:
        print('Такого клиента не существует')
        return -1
            
    # Проверяем, есть ли такой клиент 

In [188]:
def check_products(product_list):
    cur = db.cursor()
    res = []
    for a, qty in product_list:
        cur.execute('''select product_id from products where product_name = ? ''', (a, ))
        try: 
            pr_id = cur.fetchone()[0]
            res.append([pr_id, qty])
        except TypeError:
            print('Такой продукт не существует')
            return -1 
    return res

In [189]:
def add_invoice(customer_id, invoice_date='2017-04-24'):
    cur = db.cursor()
    cur.execute('''select coalesce(max(invoice_id) + 1, 1) from invoices''')
    inv_id = cur.fetchone()[0]
    cur.execute('''insert into invoices(invoice_id, customer_id, order_sum, invoice_date)
                    VALUES(?,?,NULL,?)''', (inv_id, customer_id, invoice_date))
    db.commit()
    return inv_id

In [190]:
def add_products(product_list, inv_id):
    cur = db.cursor()
    for name, qty in product_list:
        cur.execute('''insert into invoice_details(invoice_id, product_id, qty) 
                        values (?,?,?)''', (inv_id, name, qty))
    db.commit()
    return 1
    


In [191]:
def update_invoice(inv_id):
    cur = db.cursor()
    cur.execute('''
    select sum(ii.qty*price)
      from invoice_details ii
     inner join products p
             on ii.product_id = p.product_id 
     where invoice_id = ?''', (str(inv_id)))
    sum_order = cur.fetchone()[0]
    cur.execute('''Update invoices set order_sum = ? where invoice_id = ?''', (sum_order, inv_id))
    db.commit()
    

In [192]:
def add_order(client_name, product_list):
    client_id = check_client(client_name)
    if client_id==-1:
        return -1 
    prod_list = check_products(product_list)
    if prod_list == -1:
        return -2
    inv_id = add_invoice(client_id)
    add_products(prod_list, inv_id)
    update_invoice(inv_id)
    db.commit()
    

In [193]:
add_order('Google', b)

In [194]:
%%sql 
SELECT * 
from invoices

Done.


invoice_ID,customer_id,order_sum,invoice_date
1,2,260000,2017-04-24


In [195]:
%%sql
select * 
from invoice_Details

Done.


invoice_ID,product_id,qty
1,1,10
1,2,5
